# 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# 2. Dataset

In [ ]:
data = pd.read_csv('datasets/Dataset2/train.csv')
test = pd.read_csv('datasets/Dataset2/test.csv')

data.info()
data.head()
data.describe().T

# 3. Data Preprocessing

In [ ]:
# Drop id
for df in [data, test]:
    if 'id' in df.columns:
        df.drop('id', axis=1, inplace=True)

# Pisah fitur dan target
y = data['price_range']
X = data.drop('price_range', axis=1)

# Split 70:30
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Method

## 4.1. SVM

In [ ]:
svm = SVC(kernel='rbf', decision_function_shape='ovr')
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

## 4.2. MLP

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,50), max_iter=200, random_state=42)
mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

## 4.3. CNN

In [ ]:
# reshape untuk CNN-1D
timesteps = X_train.shape[1]

X_train_cnn = X_train.reshape(-1, timesteps, 1)
X_test_cnn = X_test.reshape(-1, timesteps, 1)

y_train_cat = to_categorical(y_train.astype(int))
y_test_cat = to_categorical(y_test.astype(int))

model_cnn = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(timesteps,1)),
    Dropout(0.3),
    Conv1D(64, 3, activation='relu'),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(y_train_cat.shape[1], activation='softmax')
])

model_cnn.compile('adam', 'categorical_crossentropy', ['accuracy'])
model_cnn.fit(X_train_cnn, y_train_cat, epochs=20, batch_size=32, validation_split=0.2)

# Evaluasi CNN
y_cnn_pred = np.argmax(model_cnn.predict(X_test_cnn), axis=1)
print(confusion_matrix(y_test, y_cnn_pred))

In [ ]:
probs = svm.decision_function(X_test)
print("SVM Multi-class ROC-AUC (OVR):", roc_auc_score(y_test, probs, multi_class='ovr'))